In [ ]:
import os
import glob
import flywheel
import logging
import zipfile
import platform
import subprocess
import shutil

In [ ]:
#Initialize the logger
logging.basicConfig(filename='download_mrs.log', level=logging.INFO)
logger = logging.getLogger()

In [ ]:
# Get API Key from local environment variables and initialize the Flywheel client
api_key = os.environ.get('API_KEY')
fw = flywheel.Client(api_key)

In [ ]:
# Path where the MRS data will be downloaded
root_path = ''

# Get the Flywheel Project
project = fw.lookup('')

In [ ]:
def download_all_mrs(root_data):
    #Get the group label
    group_label = project.group
    
    #Iterate through all the files in the 'ACTIVE' and 'SHAM' sessions in the project
    for subject in project.subjects():
        
        for session in subject.sessions():
            
            # Input the session labels with MRS data.
            
            if session.label == 'ACTIVE' or session.label == 'SHAM': 
                
                for acquisition in session.acquisitions():
                    
                    acq_label = acquisition.label
                    
                    #Check if the acquisition label starts with 'mrs'
                    
                    if acq_label.startswith('mrs'):
                        #Create the directory using the Flywheel hierarchy
                        path = f'{root_path}/{group_label}/{project.label}/{subject.label}/{session.label}/{acq_label}'
                        os.makedirs(path, exist_ok=True)
                        
                        for file in acquisition.files:
                            
                            #Find the dicom file in the mrs acquisition
                            if file.type == 'dicom':
                                
                                file_path = f'{path}/{file.name}'
                                        
                                file_name = f'sub-{subject.label}_ses-{session.label}_acq-{acq_label}'
                                
                                dcm_path = f'{path}/{file_name}'

                                if not os.path.exists(dcm_path):
                                    if platform.system() == 'Windows':
                                        file_path = file_path.replace('/', '\\')
                                        dcm_path = dcm_path.replace('/', '\\')
                                    try:
                                        #Download the file from the acquisition.
                                        
                                        acquisition.download_file(file.name, file_path)
                                        
                                        # Open the dicom zip file
                                        with zipfile.ZipFile(file_path, 'r') as zip_ref:
                                            #Extract the contents of the zip file to the path
                                            zip_ref.extractall(path)
                                            
                                        #Delete dicom compressed zip file
                                        os.remove(file_path)
                                        
                                        #Rename the unzipped folder to the new file name
                                        unzipped_folder = glob.glob(f'{path}/*')[0]
                                        os.rename(unzipped_folder, dcm_path)

                                        logger.info(f'Downloaded {file.name} from {acq_label} in {session.label} for {subject.label}')
                                        
                                        # convert dicoms to nifti
                                        subprocess.run(["spec2nii", "dicom", "-f", dcm_path, dcm_path])
                                        
                                        #Remove the dicoms
                                        shutil.rmtree(dcm_path)
                                        
                                        logger.info(f'Converted dicom into nifti: {file.name} from {acq_label} in {session.label} for {subject.label}')
                                        
                                    except Exception as e:
                                        logger.error(f'Error downloading {file.name} from {acq_label} in {session.label} for {subject.label}. {e}')
                                break

In [ ]:
download_all_mrs(root_path)